In [ ]:
import numpy as np
import  matplotlib.pyplot as plt
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import KFold,cross_val_score
from sklearn import preprocessing

from sklearn.model_selection import train_test_split
from collections import Counter

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
import xgboost as xgb

### only covariates

In [ ]:
dat0 = pd.read_csv('cov_mbdat.csv')

dat = dat0.sample(frac=1, random_state=2).reset_index(drop=True)
data_x=dat.drop(['xlid','y'],axis=1)
data_y=np.array(dat['y'])

cat_cols = data_x.select_dtypes(include=['object']).columns
num_cols = data_x.select_dtypes(exclude=['object']).columns
data_x_dum = pd.get_dummies(data_x, columns=cat_cols,dtype=int)
data_x=data_x_dum

xtrain,xtest,ytrain,ytest = train_test_split(data_x,data_y,test_size=0.2,random_state=222,shuffle=False)

##### Random Forest

In [ ]:
from bayes_opt import BayesianOptimization

In [ ]:
stratified_cv = StratifiedKFold(n_splits=5, shuffle=False)

In [ ]:
def rf_cv(n_estimators, min_samples_split, max_features, max_depth):
    val = cross_val_score(
        RandomForestClassifier(n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
            max_features=min(max_features, 0.999), # float
            max_depth=int(max_depth),
            random_state=666),
        data_x,  data_y, scoring='roc_auc', cv=stratified_cv,
    ).mean()
    return val

In [ ]:
rf_bo = BayesianOptimization(
        rf_cv,
        {'n_estimators': (20,25),
        'min_samples_split': (8, 15),
        'max_features': (0.8, 1),
        'max_depth': (8, 12)}
    )

In [ ]:
rf_bo.maximize()

In [ ]:
rf_bo.max

In [ ]:
rf = RandomForestClassifier(n_estimators=24, max_depth=9, max_features=0.8, min_samples_split=12,random_state=666)
np.mean(cross_val_score(rf, data_x, data_y, cv=stratified_cv, scoring='roc_auc'))

##### Xgboost

In [ ]:
from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization

In [ ]:
def xgb_cv(n_estimators, max_depth, gamma, min_child_weight):
    model = XGBClassifier(
        n_estimators=int(n_estimators),  # 注意转为整数
        max_depth=int(max_depth),
        gamma=gamma,
        min_child_weight=min_child_weight,
        random_state=666
    )
    scores = cross_val_score(model,data_x, data_y, cv=stratified_cv, scoring='roc_auc')
    return scores.mean()

xgb_bo = BayesianOptimization(
    xgb_cv,
    {
        'n_estimators': (20, 50), 
        'max_depth': (4, 8),
        'gamma': (0.3, 0.7),
        'min_child_weight': (3, 7)
    }
)
xgb_bo.maximize()

In [ ]:
xgb_bo.max

In [ ]:
xgb_model = XGBClassifier(gamma=0.3, 
                          max_depth=7, 
                          min_child_weight=5,
                          n_estimators=43,random_state=666)
np.mean(cross_val_score(xgb_model, data_x, data_y, cv=stratified_cv, scoring='roc_auc'))

In [ ]:
plt.savefig('shap_summary_plot.png', bbox_inches='tight')
plt.close()

### covariates and taxa

In [ ]:
dat0 = pd.read_csv('covtax_mbdat.csv')

dat = dat0.sample(frac=1, random_state=2).reset_index(drop=True)
data_x=dat.drop(['xlid','y'],axis=1)
data_y=np.array(dat['y'])

cat_cols = data_x.select_dtypes(include=['object']).columns
num_cols = data_x.select_dtypes(exclude=['object']).columns

data_x_dum = pd.get_dummies(data_x, columns=cat_cols,dtype=int)
data_x=data_x_dum

In [ ]:
rf_bo = BayesianOptimization(
        rf_cv,
        {'n_estimators': (70,120),
        'min_samples_split': (5, 8),
        'max_features': (0.7, 0.9),
        'max_depth': (3, 8)}
    )
rf_bo.maximize()

In [ ]:
rf_bo.max

In [ ]:
rf = RandomForestClassifier(n_estimators=97, max_depth=6, max_features=0.85, min_samples_split=5,random_state=666)
print(np.mean(cross_val_score(rf, data_x, data_y, cv=stratified_cv, scoring='roc_auc')))

In [ ]:
cv2 = StratifiedKFold(n_splits=5, shuffle=True)
print(np.mean(cross_val_score(rf, data_x, data_y, cv=cv2, scoring='roc_auc')))

In [ ]:
model = RandomForestClassifier(random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

X_base = data_x.iloc[:,14:]
X_new = data_x

y=data_y
auc_base = cross_val_score(model, X_base, y, cv=cv, scoring='roc_auc').mean()
auc_new = cross_val_score(model, X_new, y, cv=cv, scoring='roc_auc').mean()

print(f"baseAUC: {auc_base:.3f}, newAUC: {auc_new:.3f}, improve: {auc_new - auc_base:.3f}")

### without parameter optimization

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier 
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (roc_curve, auc, accuracy_score, recall_score, 
                            precision_score, f1_score, confusion_matrix)
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression

In [ ]:
dat = pd.read_csv('covtax_mbdat.csv')

data_x=dat.drop(['xlid','y'],axis=1)
data_y=np.array(dat['y'])

cat_cols = data_x.select_dtypes(include=['object']).columns
num_cols = data_x.select_dtypes(exclude=['object']).columns

data_x_dum = pd.get_dummies(data_x, columns=cat_cols,dtype=int)
data_x=data_x_dum

In [ ]:
X_base = data_x.iloc[:,14:]
X_new = data_x

y=data_y

In [ ]:
jama_colors = {
    'blue': '#357EBDFF',
    'red': '#D43F3AFF',
    'green': '#5CB85CFF',
    'orange': '#EEA236FF'
}
n_repeats = 100  
output_files = {
    'rf': "rf_100res.csv",
    'xgb': "xgb_100res.csv"
}

def initialize_results_storage():
    return {
        'results': [],
        'roc_data': {
            'base': {'tprs': [], 'aucs': []},
            'new': {'tprs': [], 'aucs': []}
        }
    }

rf_data = initialize_results_storage()
xgb_data = initialize_results_storage()

mean_fpr = np.linspace(0, 1, 100)

In [ ]:
for random_state in tqdm(range(n_repeats), desc="Processing"):
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
    
    # 初始化当前random_state的存储
    def init_fold_metrics():
        return {
            'base': {'accuracy': [], 'precision': [], 'recall': [], 
                    'specificity': [], 'f1': [], 'auc': [], 'tpr': []},
            'new': {'accuracy': [], 'precision': [], 'recall': [], 
                   'specificity': [], 'f1': [], 'auc': [], 'tpr': []}
        }
    
    rf_metrics = init_fold_metrics()
    xgb_metrics = init_fold_metrics()
    lr_metrics = init_fold_metrics()
    
    for train_idx, test_idx in cv.split(X_base, y):
        X_base_train, X_base_test = X_base.iloc[train_idx,:], X_base.iloc[test_idx,:]
        X_new_train, X_new_test = X_new.iloc[train_idx,:], X_new.iloc[test_idx,:]
        y_train, y_test = y[train_idx], y[test_idx]
        
        print("测试集负类样本数:", np.sum(y_test == 0))
        
        models = {
            'rf': RandomForestClassifier(random_state=random_state),
            'xgb': XGBClassifier(random_state=random_state)
        }
        
        for model_name, model in models.items():
            # base model（only covariate）
            model.fit(X_base_train, y_train)
            y_pred_base = model.predict(X_base_test)
            y_proba_base = model.predict_proba(X_base_test)[:, 1]
            
            # new mode (covariate + taxa)
            model.fit(X_new_train, y_train)
            y_pred_new = model.predict(X_new_test)
            y_proba_new = model.predict_proba(X_new_test)[:, 1]
            
            for features, X_test, y_pred, y_proba in zip(
                ['base', 'new'], 
                [X_base_test, X_new_test], 
                [y_pred_base, y_pred_new], 
                [y_proba_base, y_proba_new]
            ):
                tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
                metrics = rf_metrics if model_name == 'rf' else xgb_metrics
                metrics[features]['accuracy'].append(accuracy_score(y_test, y_pred))
                metrics[features]['precision'].append(precision_score(y_test, y_pred, zero_division=0))
                metrics[features]['recall'].append(recall_score(y_test, y_pred))
                metrics[features]['specificity'].append(tn / (tn + fp))
                metrics[features]['f1'].append(f1_score(y_test, y_pred))
                
                fpr, tpr, _ = roc_curve(y_test, y_proba, drop_intermediate=False)
                if fpr[0] != 0 or tpr[0] != 0:
                    fpr = np.insert(fpr, 0, 0)
                    tpr = np.insert(tpr, 0, 0)
                metrics[features]['auc'].append(auc(fpr, tpr))
                metrics[features]['tpr'].append(np.interp(mean_fpr, fpr, tpr))

    def save_results(model_data, metrics, model_name):
        record = {'random_state': random_state}
        for features in ['base', 'new']:
            for metric in ['accuracy', 'precision', 'recall', 'specificity', 'f1', 'auc']:
                key = f"{features}_{metric}"
                record[key] = np.mean(metrics[features][metric])
            
            model_data['roc_data'][features]['tprs'].append(np.mean(metrics[features]['tpr'], axis=0))
            model_data['roc_data'][features]['aucs'].append(np.mean(metrics[features]['auc']))
        
        model_data['results'].append(record)
    
    save_results(rf_data, rf_metrics, 'rf')
    save_results(xgb_data, xgb_metrics, 'xgb')

In [ ]:
pd.DataFrame(rf_data['results']).to_csv(output_files['rf'], index=False)
pd.DataFrame(xgb_data['results']).to_csv(output_files['xgb'], index=False)
print(f"RF save to {output_files['rf']}")
print(f"XGBoost save to {output_files['xgb']}")

In [ ]:
def print_summary(model_name, model_data):
    print(f"\n=== {model_name.upper()} 统计汇总 ===")
    df = pd.DataFrame(model_data['results'])
    for features in ['base', 'new']:
        print(f"\n{features.upper()} 特征:")
        for metric in ['accuracy', 'precision', 'recall', 'specificity', 'f1', 'auc']:
            col = f"{features}_{metric}"
            mean_val = df[col].mean()
            std_val = df[col].std()
            print(f"{metric:12s}: {mean_val:.3f} ± {std_val:.3f}")

print_summary('Random Forest', rf_data)
print_summary('XGBoost', xgb_data)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams

def mm_to_inch(mm):
    return mm / 25.4

plt.rcParams['font.sans-serif'] = 'Arial'
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['pdf.fonttype'] = 42  
plt.rcParams['font.size'] = 6      

nature_colors = {
    'blue': "#3C5488",    
    'red': "#E64B35",     
    'green': "#00A087",  
    'yellow': "#F39B7F",  
}

model_colors = {
    'rf': {'base': nature_colors['blue'], 'new': nature_colors['red']},
    'xgb': {'base': nature_colors['green'], 'new': nature_colors['yellow']}
}


In [ ]:
def plot_roc(model_name, model_data, colors, output_file, 
             figsize_mm=(90, 90), dpi=300):  # 默认90mm × 90mm
    figsize = (mm_to_inch(figsize_mm[0]), mm_to_inch(figsize_mm[1]))
    
    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    
    mean_fpr = np.linspace(0, 1, 100)
    
    for features in ['base', 'new']:
        tprs = []
        for tpr in model_data['roc_data'][features]['tprs']:
            interp_tpr = np.interp(mean_fpr, mean_fpr, tpr)
            interp_tpr[0] = 0.0
            tprs.append(interp_tpr)
        
        mean_tpr = np.mean(tprs, axis=0)
        mean_auc = np.mean(model_data['roc_data'][features]['aucs'])
        std_auc = np.std(model_data['roc_data'][features]['aucs'])
        
        if features == 'base':
            model_label = "10 SocDemog"
        else:
            model_label = "10 SocDemog + 14 Tax"
            
        ax.plot(mean_fpr, mean_tpr, color=colors[features],
                label=f'{model_label} (AUC = {mean_auc:.2f}±{std_auc:.2f})',
                lw=1.5, alpha=0.8)
    
    ax.plot([0, 1], [0, 1], '--', lw=1, color='gray', alpha=0.4)
    
    ax.set_xlim([-0.02, 1.02])
    ax.set_ylim([-0.02, 1.02])
    ax.set_xlabel('False Positive Rate', fontsize=6)
    ax.set_ylabel('True Positive Rate', fontsize=6)
    ax.set_title(f'{model_name} ROC Curves', fontsize=7, pad=4)
    
    ax.legend(loc="lower right", fontsize=5, framealpha=0.5)
    ax.grid(True, linestyle=':', alpha=0.3)
    
    plt.savefig(output_file, dpi=dpi, bbox_inches='tight', 
                pad_inches=0.05, transparent=True)
    plt.close()



In [ ]:
plot_roc('Random Forest', rf_data, 
         colors=model_colors['rf'],
         output_file='RF_ROC.pdf',
         figsize_mm=(87.5, 60))  # 90mm × 90mm

plot_roc('XGBoost', xgb_data,
         colors=model_colors['xgb'],
         output_file='XGB_ROC.pdf',
         figsize_mm=(87.5, 60))  # 90mm × 60mm

In [ ]:
import shap
maxr=8

rf_model=RandomForestClassifier(random_state=maxr)
xgb_model=XGBClassifier(random_state=maxr)

rf_model.fit(X_new, y)
xgb_model.fit(X_new, y)


explainer1 = shap.Explainer(rf_model, X_new)  
explainer2 = shap.Explainer(xgb_model, X_new) 

shap_values1 = explainer1(X_new)
shap_values2 = explainer2(X_new)

In [ ]:
shap.plots.bar(shap_values[:, :, 1], max_display=21)

In [ ]:
if len(shap_values.shape) == 3:
    shap_values = shap_values[:, :, 1]
shap.plots.bar(shap_values.mean(0), max_display=21)  # 显示前20个重要特征
plt.close()

In [ ]:
# plt.savefig('shap_importance_plot.png', bbox_inches='tight')
#if，RF 
#shap.plots.bar(shap_values[:, :, 1], max_display=21)  # 显示前20个重要特征
#if，XGB
shap.plots.bar(shap_values.mean(0), max_display=21)  # 显示前20个重要特征
plt.close()

In [ ]:
#shap.summary_plot(shap_values[..., 1], X_new, plot_type="dot")  # RF
shap.summary_plot(shap_values, X_new, plot_type="dot")  # XGB

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.collections import PathCollection 
from matplotlib import rcParams
import shap

rcParams['font.sans-serif'] = 'Arial' 
rcParams['font.size'] = 6            
rcParams['pdf.fonttype'] = 42         

fig_width_mm = 45  # 
fig_height_mm = 120 # 
figsize = (mm_to_inch(fig_width_mm), mm_to_inch(fig_height_mm))

plt.figure(figsize=figsize) 

shap.summary_plot(
    shap_values1[..., 1],         
    X_new,                  
    plot_type="dot",             
    show=False,                   
    plot_size=None,               
    max_display=20               
)

ax = plt.gca()  

for collection in ax.collections:
    if isinstance(collection, PathCollection):
        collection.set_sizes([3]) 

for text in ax.texts[:]:
    if 'High' in text.get_text().lower() or 'Low' in text.get_text().lower():
        text.remove()
    
ax.set_xlabel('SHAP value (Random Forest)', fontsize=6) 
ax.tick_params(axis='both', labelsize=5) 

plt.savefig(
    'rfshap_top20.pdf',  
    dpi=300,
    bbox_inches='tight',     
    pad_inches=0.05,        
    transparent=True        
)
plt.close()

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.collections import PathCollection
from matplotlib import rcParams
import shap

rcParams['font.sans-serif'] = 'Arial'  
rcParams['font.size'] = 6             
rcParams['pdf.fonttype'] = 42        

fig_width_mm = 45  # 
fig_height_mm = 120 # 
figsize = (mm_to_inch(fig_width_mm), mm_to_inch(fig_height_mm))

plt.figure(figsize=figsize)  

shap.summary_plot(
    shap_values2,         
    X_new,                 
    plot_type="dot",             
    show=False,                 
    plot_size=None,              
    max_display=20               
)

ax = plt.gca() 

for collection in ax.collections:
    if isinstance(collection, PathCollection): 
        collection.set_sizes([3])  

for text in ax.texts[:]:
    if 'High' in text.get_text().lower() or 'Low' in text.get_text().lower():
        text.remove()
    
ax.set_xlabel('SHAP value (XGBoost)', fontsize=6)
ax.tick_params(axis='both', labelsize=5) 

plt.savefig(
    'xgbshap_top20.pdf',  
    dpi=300,
    bbox_inches='tight',     
    pad_inches=0.05,         
    transparent=True      
)
plt.close() 